<a href="https://colab.research.google.com/github/daoyul1/DL-LLMScience-RAG/blob/main/Generate_FAISS_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [2]:
pip install --no-cache-dir faiss-gpu==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 184.7 MB/s eta 0:00:00


In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Load Model and Libraries

Used BAAI/bge-small-en-v1.5 model for generating faiss indexes.

In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datasets import load_from_disk

# Load bge-small-en-v1.5 model
model = SentenceTransformer("BAAI/bge-small-en-v1.5", device="cuda")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Download Preprocessed Wikipedia Dataset from Kaggle

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mbanaei/all-paraphs-parsed-expanded")

100%|██████████| 524M/524M [00:05<00:00, 108MB/s]

Extracting files...


### Load and Prepare Dataset

In [6]:
# Load the preprocessed Wikipedia dataset
dataset = load_from_disk(path)

# Extract text chunks
texts = [row["text"] for row in dataset]

In [13]:
import torch
torch.cuda.empty_cache()  # Clear unused GPU memory

### Generate Embeddings

In [14]:
# Generate embeddings with bge-1.5
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True, device="cuda")
embeddings = np.array(embeddings, dtype=np.float32)  # FAISS requires float32

Batches:   0%|          | 0/32833 [00:00<?, ?it/s]

### Build & Save FAISS Index

In [15]:
# Initialize FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance for 384 dimensions
index.add(embeddings)

# Save the FAISS index
faiss.write_index(index, "bge-small-faiss.index")
print("FAISS index created with bge-small-en-v1.5.")

FAISS index created with bge-small-en-v1.5.


### Test the FAISS Index

In [16]:
query = ["What is the theory of relativity?"]

# Generate query embedding
query_embedding = model.encode(query, device="cuda")
query_embedding = np.array(query_embedding, dtype=np.float32)

# Search the FAISS index
distances, indices = index.search(query_embedding, k=5)
print("Top-5 closest matches:", indices)

Top-5 closest matches: [[1486907 1312782 1573156  916173  907733]]


### Generating FAISS Index by using all-mpnet-base-v2

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datasets import load_from_disk

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="cuda")

dataset = load_from_disk(path)

# Extract text chunks
texts = [row["text"] for row in dataset]

# Generate embeddings with the new model
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True, device="cuda")
embeddings = np.array(embeddings, dtype=np.float32)  # FAISS requires float32

# Initialize FAISS index (using L2 for normalized embeddings)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Save the FAISS index
faiss.write_index(index, "all-mpnet-base-v2-faiss.index")
print("FAISS index created with all-mpnet-base-v2.")

# Inference-like retrieval process:
query = ["What is the theory of relativity?"]
query_embedding = model.encode(query, device="cuda")
query_embedding = np.array(query_embedding, dtype=np.float32)

# Search the FAISS index
distances, indices = index.search(query_embedding, k=5)
print("Top-5 closest matches:", indices)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32833 [00:00<?, ?it/s]

FAISS index created with all-mpnet-base-v2.
Top-5 closest matches: [[1486907  907733  907729  916173 1273101]]
